# Covid SQL Data Analysis

The objective of this workbook is to use SQL to analyze a create tables for Tableau visualization regarding Covid-19. For the simplicity of running this on a personal computer, we will be using SQLite. 

To begin, we have started by retrieving global Covid-19 data from the following link: 
https://ourworldindata.org/covid-deaths

Using DB Browser for SQLite, we created a .db file to explore using the data from the above link. 

The below queries are intended to generate tables & csv files that can be explored visually with Tableau. Link to Tableau Covid-19 Visualization here: https://public.tableau.com/views/CovidAugustReview/Story1?:language=en-US&:display_count=n&:origin=viz_share_link


In [1]:
%%capture
%load_ext sql
%sql sqlite:///CovidData.db

In [2]:
%%sql
-- Overview of current database tables 

Select * 
FROM sqlite_master 
WHERE type='table';

 * sqlite:///CovidData.db
Done.


type,name,tbl_name,rootpage,sql
table,coviddeaths,coviddeaths,8300,"CREATE TABLE ""coviddeaths"" ( ""iso_code"" TEXT, ""continent"" TEXT, ""location"" TEXT, ""date"" TEXT, ""population"" INTEGER, ""total_cases"" INTEGER, ""new_cases"" INTEGER, ""new_cases_smoothed"" REAL, ""total_deaths"" INTEGER, ""new_deaths"" INTEGER, ""new_deaths_smoothed"" REAL, ""total_cases_per_million"" TEXT, ""new_cases_per_million"" REAL, ""new_cases_smoothed_per_million"" REAL, ""total_deaths_per_million"" REAL, ""new_deaths_per_million"" REAL, ""new_deaths_smoothed_per_million"" REAL, ""reproduction_rate"" REAL, ""icu_patients"" INTEGER, ""icu_patients_per_million"" REAL, ""hosp_patients"" INTEGER, ""hosp_patients_per_million"" REAL, ""weekly_icu_admissions"" INTEGER, ""weekly_icu_admissions_per_million"" REAL, ""weekly_hosp_admissions"" INTEGER, ""weekly_hosp_admissions_per_million"" REAL)"
table,covidvax,covidvax,3445,"CREATE TABLE ""covidvax"" ( ""iso_code"" TEXT, ""continent"" TEXT, ""location"" TEXT, ""date"" TEXT, ""new_tests"" INTEGER, ""total_tests"" INTEGER, ""total_tests_per_thousand"" REAL, ""new_tests_per_thousand"" REAL, ""new_tests_smoothed"" REAL, ""new_tests_smoothed_per_thousand"" REAL, ""positive_rate"" REAL, ""tests_per_case"" REAL, ""tests_units"" INTEGER, ""total_vaccinations"" INTEGER, ""people_vaccinated"" INTEGER, ""people_fully_vaccinated"" INTEGER, ""total_boosters"" INTEGER, ""new_vaccinations"" INTEGER, ""new_vaccinations_smoothed"" REAL, ""total_vaccinations_per_hundred"" REAL, ""people_vaccinated_per_hundred"" REAL, ""people_fully_vaccinated_per_hundred"" REAL, ""total_boosters_per_hundred"" REAL, ""new_vaccinations_smoothed_per_million"" REAL, ""stringency_index"" REAL, ""population_density"" REAL, ""median_age"" REAL, ""aged_65_older"" REAL, ""aged_70_older"" REAL, ""gdp_per_capita"" REAL, ""extreme_poverty"" REAL, ""cardiovasc_death_rate"" REAL, ""diabetes_prevalence"" REAL, ""female_smokers"" REAL, ""male_smokers"" REAL, ""handwashing_facilities"" REAL, ""hospital_beds_per_thousand"" REAL, ""life_expectancy"" REAL, ""human_development_index"" REAL, ""excess_mortality"" REAL)"


In [3]:
%%sql

-- Query to look at Deaths vs Location 


DROP TABLE IF EXISTS Cases_Deaths_by_Location_Date; 
CREATE TABLE Cases_Deaths_by_Location_Date AS 
    SELECT Location, date, total_cases, new_cases, total_deaths, population
    FROM coviddeaths
    ORDER BY 1,2;
    


 * sqlite:///CovidData.db
Done.
Done.


[]

In [4]:
%%sql

-- Query to look at Death Rate by Location, by Date
-- Introducing Death_Rate column 

DROP TABLE IF EXISTS Death_Rate_by_Location_Date; 
CREATE TABLE Death_Rate_by_Location_Date AS 
    SELECT Location, date, total_cases, total_deaths, 
        CAST(total_deaths AS FLOAT)/total_cases*100 as Death_Rate, population
    FROM coviddeaths
    ORDER BY 1,2
    LIMIT 40;



 * sqlite:///CovidData.db
Done.
Done.


[]

In [5]:
%%sql

-- Query to look at Total Cases vs Population
-- Cast total_cases column as Float type to avoid int / int error  

DROP TABLE IF EXISTS Case_Rate_per_Capita; 
CREATE TABLE Case_Rate_per_Capita AS 
    SELECT Location, date, total_cases, Population, 
        CAST(total_cases AS FLOAT)/population*100 as Case_Rate
    FROM coviddeaths
    ORDER BY 1,2;


 * sqlite:///CovidData.db
Done.
Done.


[]

In [6]:
%%sql

-- Query to look at Countries with highest infection rates vs population
-- 

DROP TABLE IF EXISTS Infection_Rate_per_Capita; 
CREATE TABLE Infection_Rate_per_Capita AS 
    SELECT Location, Population, max(CAST(total_cases AS INT)) as Max_Cases_Count, 
        MAX((CAST(total_cases AS FLOAT)/population)*100) as Infection_Rate_Per_Capita
    FROM coviddeaths
    GROUP BY Location, Population
    ORDER BY Infection_Rate_Per_Capita DESC;

 * sqlite:///CovidData.db
Done.
Done.


[]

In [7]:
%%sql

-- Query to look at each location's death count vs population
-- Alternate approach - multiple column by float (1.0) to avoid int / int error
-- Filtering locations that have no reported death counts (assuming missing info)
-- Filering out previously aggregated locations to avoid double counting ("WHERE continent IS NOT '' ")
    -- Ex) Filters out locations such as "Europe" etc. 


DROP TABLE IF EXISTS Death_Rate_per_Capita; 
CREATE TABLE Death_Rate_per_Capita AS 
    SELECT Location, continent, Population, MAX(total_deaths*1) as Death_Count, 
        MAX((((total_deaths *1.0))/population)*100) as Death_Rate_Per_Capita
    FROM coviddeaths
    WHERE continent IS NOT ''
    GROUP BY Location, Population
    HAVING Death_Count >0
    ORDER BY Death_Count DESC;

 * sqlite:///CovidData.db
Done.
Done.


[]

In [8]:
%%sql

-- Query to look at deaths on each continent
-- Filtering out certain results to avoid double counting (e.g. "European Union")

DROP TABLE IF EXISTS Deaths_by_Continent; 
CREATE TABLE Deaths_by_Continent AS 
    SELECT location,  MAX(total_deaths*1) as Death_count
    FROM coviddeaths
    WHERE continent IS '' and location is not 'World' and location is not 'European Union'
    GROUP BY location
    ORDER BY Death_count DESC;



 * sqlite:///CovidData.db
Done.
Done.


[]

In [9]:
%%sql

-- Query to look at global cases by date
-- Using windows function to calculate running total of cases with "new_cases" column 

DROP TABLE IF EXISTS Global_Cases_RT;
CREATE TABLE Global_Cases_RT (date, Running_total_cases);

WITH NewCasesByDate as (    
    SELECT date,  sum(new_cases) as total_new_cases
    FROM coviddeaths
    WHERE continent is not ''
    GROUP BY date
    ORDER BY date
    )
INSERT INTO Global_Cases_RT
SELECT date, sum(total_new_cases) OVER (ORDER BY (date)) as Running_total_cases
from NewCasesByDate
ORDER BY date;




 * sqlite:///CovidData.db
Done.
Done.
Done.


[]

In [10]:
%%sql 

-- Query to look at new and total vaccines by day globally
-- Inner join tables on date and location columns

DROP TABLE IF EXISTS Global_Vax_by_date;
CREATE TABLE Global_Vax_by_date AS
SELECT cd.location Location, cd.date Date, cd.population Population, cv.total_vaccinations, cv.new_vaccinations
FROM coviddeaths as cd
INNER JOIN covidvax as cv
    ON cd.date = cv.date
    AND cd.location = cv.location
WHERE cd.Location = 'World'
ORDER BY Date;



 * sqlite:///CovidData.db
Done.
Done.


[]

In [11]:
%%sql 

-- Query to look at new and total vaccines by day in Canada
-- Inner join tables on date and location columns

DROP TABLE IF EXISTS CAN_Vax_by_date;
CREATE TABLE CAN_Vax_by_date AS
SELECT cd.location Location, 
        cd.date Date, 
        cd.population Population, 
        cv.total_vaccinations, 
        cv.people_vaccinated, 
        cv.people_fully_vaccinated,
        cv.new_vaccinations, 
        cd.total_cases, 
        cd.new_cases        
FROM coviddeaths as cd
INNER JOIN covidvax as cv
    ON cd.date = cv.date
    AND cd.location = cv.location
WHERE cd.Location = 'Canada'
ORDER BY Date;




 * sqlite:///CovidData.db
Done.
Done.


[]

In [12]:
%%sql
-- Overview of database now that we have added to it

Select * 
FROM sqlite_master 
WHERE type='table';

 * sqlite:///CovidData.db
Done.


type,name,tbl_name,rootpage,sql
table,coviddeaths,coviddeaths,8300,"CREATE TABLE ""coviddeaths"" ( ""iso_code"" TEXT, ""continent"" TEXT, ""location"" TEXT, ""date"" TEXT, ""population"" INTEGER, ""total_cases"" INTEGER, ""new_cases"" INTEGER, ""new_cases_smoothed"" REAL, ""total_deaths"" INTEGER, ""new_deaths"" INTEGER, ""new_deaths_smoothed"" REAL, ""total_cases_per_million"" TEXT, ""new_cases_per_million"" REAL, ""new_cases_smoothed_per_million"" REAL, ""total_deaths_per_million"" REAL, ""new_deaths_per_million"" REAL, ""new_deaths_smoothed_per_million"" REAL, ""reproduction_rate"" REAL, ""icu_patients"" INTEGER, ""icu_patients_per_million"" REAL, ""hosp_patients"" INTEGER, ""hosp_patients_per_million"" REAL, ""weekly_icu_admissions"" INTEGER, ""weekly_icu_admissions_per_million"" REAL, ""weekly_hosp_admissions"" INTEGER, ""weekly_hosp_admissions_per_million"" REAL)"
table,covidvax,covidvax,3445,"CREATE TABLE ""covidvax"" ( ""iso_code"" TEXT, ""continent"" TEXT, ""location"" TEXT, ""date"" TEXT, ""new_tests"" INTEGER, ""total_tests"" INTEGER, ""total_tests_per_thousand"" REAL, ""new_tests_per_thousand"" REAL, ""new_tests_smoothed"" REAL, ""new_tests_smoothed_per_thousand"" REAL, ""positive_rate"" REAL, ""tests_per_case"" REAL, ""tests_units"" INTEGER, ""total_vaccinations"" INTEGER, ""people_vaccinated"" INTEGER, ""people_fully_vaccinated"" INTEGER, ""total_boosters"" INTEGER, ""new_vaccinations"" INTEGER, ""new_vaccinations_smoothed"" REAL, ""total_vaccinations_per_hundred"" REAL, ""people_vaccinated_per_hundred"" REAL, ""people_fully_vaccinated_per_hundred"" REAL, ""total_boosters_per_hundred"" REAL, ""new_vaccinations_smoothed_per_million"" REAL, ""stringency_index"" REAL, ""population_density"" REAL, ""median_age"" REAL, ""aged_65_older"" REAL, ""aged_70_older"" REAL, ""gdp_per_capita"" REAL, ""extreme_poverty"" REAL, ""cardiovasc_death_rate"" REAL, ""diabetes_prevalence"" REAL, ""female_smokers"" REAL, ""male_smokers"" REAL, ""handwashing_facilities"" REAL, ""hospital_beds_per_thousand"" REAL, ""life_expectancy"" REAL, ""human_development_index"" REAL, ""excess_mortality"" REAL)"
table,Cases_Deaths_by_Location_Date,Cases_Deaths_by_Location_Date,15,"CREATE TABLE Cases_Deaths_by_Location_Date( location TEXT, date TEXT, total_cases INT, new_cases INT, total_deaths INT, population INT)"
table,Death_Rate_by_Location_Date,Death_Rate_by_Location_Date,2,"CREATE TABLE Death_Rate_by_Location_Date( location TEXT, date TEXT, total_cases INT, total_deaths INT, Death_Rate, population INT)"
table,Case_Rate_per_Capita,Case_Rate_per_Capita,3,"CREATE TABLE Case_Rate_per_Capita( location TEXT, date TEXT, total_cases INT, population INT, Case_Rate)"
table,Infection_Rate_per_Capita,Infection_Rate_per_Capita,5,"CREATE TABLE Infection_Rate_per_Capita( location TEXT, population INT, Max_Cases_Count, Infection_Rate_Per_Capita)"
table,Death_Rate_per_Capita,Death_Rate_per_Capita,7,"CREATE TABLE Death_Rate_per_Capita( location TEXT, continent TEXT, population INT, Death_Count, Death_Rate_Per_Capita)"
table,Deaths_by_Continent,Deaths_by_Continent,9,"CREATE TABLE Deaths_by_Continent( location TEXT, Death_count)"
table,Global_Cases_RT,Global_Cases_RT,10,"CREATE TABLE Global_Cases_RT (date, Running_total_cases)"
table,Global_Vax_by_date,Global_Vax_by_date,13,"CREATE TABLE Global_Vax_by_date( Location TEXT, Date TEXT, Population INT, total_vaccinations INT, new_vaccinations INT)"


Let's output our tables to individual CSV files so that we can visualize them with Tableau Public (free version does not allow for direct database integration). 

In [13]:
import pandas as pd
import sqlite3

# connect with DB
conn = sqlite3.connect('CovidData.db')
c = conn.cursor()

for table in c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall():
    t = table[0]
    df = pd.read_sql('SELECT * from ' + t, conn)
    df.to_csv(t + '.csv', index = False)
